# TP2 - Feature Engineering


In [1]:
%pip install nltk #scikit-learn
%pip install scikit-learn==1.0.2
%pip install pandas numpy

import os
import pandas as pd
import numpy as np
#import seaborn as sns
#from scipy.stats import chi2_contingency
#import matplotlib.pyplot as plt
#import nltk
from nltk.corpus import stopwords
import time

import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score #, classification_report
from sklearn.neighbors import KNeighborsClassifier


Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 4.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.2 requires scikit-learn>=1.3.1, but you have scikit-learn 1.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Étape 1 : On considère les fichiers du répertoire bbcsport
* Indiquer les points marquants l'exploration.
* Pour chaque observation, indiquer l’opération à effectuer qui serait la plus appropriée.

In [2]:
start_time = time.time()

# Définir le chemin du répertoire principal
base_dir = "./bbcsport"

# Initialiser une liste pour stocker les données
data = []

# Parcourir chaque sous-répertoire
for category in os.listdir(base_dir):
    category_path = os.path.join(base_dir, category)
    
    # Vérifier si c'est bien un répertoire
    if os.path.isdir(category_path):
        # Lire tous les fichiers dans le sous-répertoire
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            
            # Lire le contenu du fichier texte
            with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                text = file.read()
                
            # Ajouter aux données
            data.append((text, category))
    


# Créer le DataFrame
df = pd.DataFrame(data, columns=["texte", "categorie"])

#from sklearn.utils import resample

df = df.reset_index()
#df.rename(columns={"index": "ID"}, inplace=True)

print('Dimensions:',df.shape, '\n')

maxCharacters = 100
pd.set_option("display.max_colwidth", maxCharacters)  # Affiche jusqu'à maxCharacters caractères

# Afficher les 20 premières lignes du DataFrame avec retours à la ligne
print('Dataframe original:')
display(df.head(20))

# Mélanger les lignes du DataFrame de façon aléatoire
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


Dimensions: (737, 3) 

Dataframe original:


,index,texte,categorie
0,0,"Van Nistelrooy hungry for return\n\nManchester United striker Ruud van Nistelrooy said he was ""h...",football
1,1,Reyes tricked into Real admission\n\nJose Antonio Reyes has added to speculation linking him wit...,football
2,2,Adriano's Chelsea link rejected\n\nAdriano's agent Gilmar Rinaldi has insisted that he has had n...,football
3,3,Owen determined to stay in Madrid\n\nEngland forward Michael Owen has told the BBC he is happy i...,football
4,4,Souness backs Smith for Scotland\n\nGraeme Souness believes Walter Smith would be the perfect ch...,football
5,5,Klinsmann issues Lehmann warning\n\nGermany coach Jurgen Klinsmann has warned goalkeeper Jens Le...,football
6,6,McClaren hails Boro's Uefa spirit\n\nMiddlesbrough boss Steve McClaren has praised the way his s...,football
7,7,Benitez delight after crucial win\n\nLiverpool manager Rafael Benitez admitted victory against D...,football
8,8,Juninho demand for O'Neill talks\n\nJuninho's agent has confirmed that the player is hoping for ...,football
9,9,Gallas sees two-horse race\n\nChelsea's William Gallas believes they will battle it out with Ars...,football


## Rebalancer le dataset

In [3]:
%pip install imblearn

from imblearn.over_sampling import SMOTE

# Initialiser TF-IDF
vectorizer = TfidfVectorizer(max_features=500)  # Limite à 500 features pour éviter trop de sparsité
X_tfidf = vectorizer.fit_transform(df['texte']).toarray()  # Transformer en matrice dense

# Labels
y = df['categorie']

# Appliquer SMOTE
# Génère artificiellement des exemples en interpolant des données existantes.
# Équilibre toutes les catégories en ajoutant des points synthétiques.
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Reconstruction du DataFrame
from sklearn.neighbors import NearestNeighbors

# Reconvertir en texte en utilisant les plus proches voisins
nn = NearestNeighbors(n_neighbors=1).fit(X_tfidf)
_, indices = nn.kneighbors(X_resampled)

# Associer chaque vecteur généré à un texte proche existant
df = pd.DataFrame({
    'texte': df['texte'].iloc[indices.flatten()].values,  # Texte approximé
    'categorie': y_resampled
})
print(df['categorie'].value_counts())  # Vérifier l'équilibrage
print('catégories:',df['categorie'].unique())  # Vérifier les categoris uniques

  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.
tennis       265
football     265
rugby        265
cricket      265
athletics    265
Name: categorie, dtype: int64
catégories: ['tennis' 'football' 'rugby' 'cricket' 'athletics']


### Extraction des mots-clés par catégorie et appliquer des prédicteurs de base

In [4]:
%pip install spacy
!python -m spacy download en_core_web_sm
import spacy

# Charger le modèle NLP anglais pour l'analyse linguistique (verbes, noms, etc.)
nlp = spacy.load("en_core_web_sm")

# Définir les catégories sportives
categories = ['tennis', 'athletics', 'rugby', 'cricket', 'football']

# Définir les mots-clés spécifiques à chaque sport
keywords = {
    "tennis": ["serve", "racket", "wimbledon", "grand slam", "ace", "match", "court"],
    "athletics": ["100m", "hurdles", "relay", "track", "marathon", "jump", "sprint"],
    "rugby": ["scrum", "try", "tackle", "ruck", "conversion", "union", "league"],
    "cricket": ["innings", "bowler", "batsman", "wicket", "yorker", "over", "stump"],
    "football": ["goal", "penalty", "striker", "midfield", "defender", "league", "cup"]
}

# Fusionner tous les mots-clés
all_keywords = list(set([word for sublist in keywords.values() for word in sublist]))

def extract_features(text):
    doc = nlp(text.lower())  # Convertir en minuscule pour l'uniformité

    # Prédicteurs structurels
    num_chars = len(text)
    num_words = len(text.split())
    num_sentences = text.count('.') + text.count('!') + text.count('?')
    avg_word_length = np.mean([len(word) for word in text.split()]) if num_words > 0 else 0
    avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
    num_short_words = sum(1 for word in text.split() if len(word) <= 3)
    num_long_words = sum(1 for word in text.split() if len(word) >= 7)

    # Charger la liste des stopwords en anglais
    stop_words = set(stopwords.words("english"))

    # Fonction pour compter les stopwords dans un texte
    words = text.split()  # Découper en mots
    num_stop_words = sum(1 for word in words if word.lower() in stop_words)
    
    # Mots-clés spécifiques aux sports
    keyword_counts = [text.lower().count(word) for word in all_keywords]

    # Fréquences lexicales et linguistiques
    num_unique_words = len(set(text.split()))
    lexical_richness = num_unique_words / num_words if num_words > 0 else 0
    num_verbs = sum(1 for token in doc if token.pos_ == "VERB")
    num_nouns = sum(1 for token in doc if token.pos_ == "NOUN")
    num_digits = sum(1 for char in text if char.isdigit())
    num_uppercase = sum(1 for char in text if char.isupper())

    # NER: Compter les entités pertinentes
    num_athletes = sum(1 for ent in doc.ents if ent.label_ in ["PERSON"])  # Compte les noms d'athlètes (ex: Usain Bolt, Roger Federer).
    num_competitions = sum(1 for ent in doc.ents if ent.label_ in ["EVENT"]) # Compte les compétitions (ex: Wimbledon, Olympic Games).
    num_locations = sum(1 for ent in doc.ents if ent.label_ in ["GPE", "LOC"]) # Compte les lieux géographiques (ex: Paris, Manchester).
    num_proper_nouns = sum(1 for token in doc if token.pos_ == "PROPN") # Compte les noms propres (ex: "Messi", "Ronaldo").
    
    # Caractères spécifiques et ponctuation
    num_punctuation = sum(1 for char in text if char in ".,!?")
    num_hashtags = text.count("#")
    num_mentions = text.count("@")

    return [
        num_chars, num_words, num_sentences, num_stop_words, avg_word_length, avg_sentence_length,
        num_short_words, num_long_words, num_unique_words, lexical_richness,
        num_verbs, num_nouns, num_digits, num_uppercase, num_punctuation,
        num_hashtags, num_mentions
    ] + keyword_counts

# Appliquer la fonction à la colonne 'texte'
df_features = df['texte'].apply(lambda x: extract_features(str(x)))
feature_names = [
    "num_chars", "num_words", "num_sentences", "num_stop_words", "avg_word_length", "avg_sentence_length",
    "num_short_words", "num_long_words", "num_unique_words", "lexical_richness",
    "num_verbs", "num_nouns", "num_digits", "num_uppercase", "num_punctuation",
    "num_hashtags", "num_mentions"
] + all_keywords  # Ajouter les mots-clés aux colonnes

df_features_init = pd.DataFrame(df_features.tolist(), columns=feature_names)

# Ajouter TF-IDF des mots les plus discriminants
vectorizer = TfidfVectorizer(stop_words='english', max_features=10)  # Garder les 10 meilleurs mots
tfidf_matrix = vectorizer.fit_transform(df['texte'])
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concaténer toutes les features
df_features_base = pd.concat([df_features_init, df_tfidf], axis=1)
df = pd.concat([df, df_features_base], axis=1)
display(df.head())  # Afficher les premières lignes du DataFrame final

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.21.0-py3-none-any.whl.metadata (14 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 4.5 MB/s eta 0:00:00a 0:00:01
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached langcodes-3.5.0-py3-none-any.whl (182 kB)
Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
   ━━━━━━━━━━━━

,texte,categorie,num_chars,num_words,num_sentences,num_stop_words,avg_word_length,avg_sentence_length,num_short_words,num_long_words,...,england,game,new,said,second,team,time,win,world,year
0,Henman overcomes rival Rusedski\n\nTim Henman saved a match point before fighting back to defeat...,tennis,1886,340,14,135,4.538235,24.285714,136,62,...,0.000000,0.615147,0.0,0.141883,0.432910,0.436803,0.379040,0.200123,0.198833,0.000000
1,Ferguson urges Henry punishment\n\nSir Alex Ferguson has called on the Football Association to p...,football,1622,273,16,114,4.930403,17.062500,110,73,...,0.000000,0.713502,0.0,0.493707,0.000000,0.000000,0.439645,0.232121,0.000000,0.000000
2,"Fuming Robinson blasts officials\n\nEngland coach Andy Robinson insisted he was ""livid"" after hi...",rugby,2141,386,25,169,4.536269,15.440000,147,74,...,0.619463,0.684660,0.0,0.118437,0.000000,0.000000,0.000000,0.334107,0.000000,0.147821
3,Arsenal through on penalties\n\nArsenal win 4-2 on penalties\n\nThe Spanish goalkeeper saved fro...,football,3138,508,27,178,5.155512,18.814815,157,151,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.905024,0.318553,0.000000,0.281878
4,Man City 1-1 Newcastle\n\nAlan Shearer hit his 250th Premiership goal to help Newcastle earn a b...,football,3850,666,36,258,4.771772,18.500000,235,149,...,0.000000,0.426380,0.0,0.000000,0.675148,0.000000,0.394090,0.416138,0.000000,0.184114


### Encodage de la cible

In [5]:
from sklearn.preprocessing import LabelEncoder

# Définir les labels de sport (catégories)
categories = ['tennis', 'athletics', 'rugby', 'cricket', 'football']
le = LabelEncoder()  # Pour encoder les catégories de sport sous forme numérique
df['category_encoded'] = le.fit_transform(df['categorie'])  # 'category' est la colonne avec les labels de sport

# Séparer les features et les labels
X = df_features_base  # Les features extraites
y = df['category_encoded']  # Les labels encodés


## Profilage

In [7]:
%pip install ipywidgets
%pip install sweetviz ydata-profiling
# %pip uninstall -y numpy #ydata-profiling



import sweetviz as sv
import warnings

# Ignore all deprecation warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

df_profiled = pd.concat([df['category_encoded'], df_features_base], axis=1)

report = sv.analyze(df_profiled)
report.show_html("profile_sortie.html")

Note: you may need to restart the kernel to use updated packages.
  Using cached sweetviz-2.3.1-py3-none-any.whl.metadata (24 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
Using cached sweetviz-2.3.1-py3-none-any.whl (15.1 MB)
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


                                             |          | [  0%]   00:00 -> (? left)

Report profile_sortie.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Sélection de Prédicteurs par Random Forest (RF)

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Entraîner le modèle Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Récupérer l'importance des features
importances = rf.feature_importances_
sorted_indices = np.argsort(importances)[::-1]  # Tri décroissant
print(len(sorted_indices))

# Sélectionner les 20 meilleures features
feature_names = df_features_base.columns
top_k = 20
best_features = [feature_names[i] for i in sorted_indices[:top_k]]

# Créer un nouveau DataFrame X_rfe avec ces features
X_rfe = X[best_features]

# Affichage des features sélectionnées
print("Top 20 features sélectionnées:", best_features)

61
Top 20 features sélectionnées: ['england', 'wicket', 'sprint', 'num_digits', 'world', 'avg_word_length', 'match', 'game', 'goal', 'batsman', 'league', 'bowler', 'innings', 'num_long_words', 'avg_sentence_length', 'num_nouns', 'said', 'num_uppercase', 'lexical_richness', 'wimbledon']


## Etape 3: Nettoyage des données

In [9]:
# Prétraitement du texte
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Suppression des ponctuations
    text = re.sub("\d+", "", text)  # Suppression des chiffres
    return text

df["cleaned_text"] = df["texte"].apply(clean_text)

# Séparation des données X et y
X = df["cleaned_text"]
y = df["categorie"]

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

df_metrics = pd.DataFrame(columns=["Technique", "Modele", "Score F1", "Accuracy"])

## Étape 4: Utilisation d'une technique d’extraction (par ex: TF–IDF Vectors) et choisir au moins 3 algorithmes de classification. De ce fait, on obtient au moins 3 modèles. On fera la comparaison en se basant la technique d’extraction.

In [10]:
# Modèles de classification
models = {
    "Logistic Regression": LogisticRegression(max_iter=10000, solver="lbfgs"),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier()
}

# Entraînement et évaluation
def appliquer_model_classification(technique, train_x, test_x):
    results = {}
    for name, model in models.items():
        model.fit(train_x, y_train)
        y_pred = model.predict(test_x)
        f1 = f1_score(y_test, y_pred, average='weighted')
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = {"F1-score": f1, "Accuracy": accuracy}
        df_metrics.loc[len(df_metrics)] = [technique, name, f1, accuracy]
        #print(technique, ',  ', name)
        #print(classification_report(y_test, y_pred))

    # Affichage des résultats
    print("\nComparaison des scores F1 et Accuracy:")
    for model, scores in results.items():
        print(f"{model}: F1-score = {scores['F1-score']:.4f}, Accuracy = {scores['Accuracy']:.4f}")

In [11]:
from collections import Counter
from scipy.sparse import hstack

all_words = " ".join(df["cleaned_text"]).split()
word_freq = Counter(all_words)
most_common_words = [word for word, _ in word_freq.most_common(2000)]  # Top 2000 mots
print('most common words:', most_common_words[:100]) #  Affiche les 100 premiers éléments de la liste.
techniques = {
    "TfidfVectorizer": TfidfVectorizer(),
    "Bag of Words": CountVectorizer(),
    "Word Embeddings": None,
    "NLP based features": None,
    "Prédicteurs de base": CountVectorizer(vocabulary=most_common_words),
    "Prédicteurs de base sélectionnés": None
}

most common words: ['the', 'to', 'in', 'a', 'and', 'of', 'for', 'was', 'on', 'he', 'i', 'but', 'is', 'with', 'that', 'at', 'his', 'it', 'have', 'as', 'be', 'has', 'said', 'will', 'we', 'from', 'not', 'by', 'after', 'had', 'an', 'who', 'first', 'they', 'their', 'been', 'are', 'out', 'england', 'this', 'm', 'world', 'over', 'when', 'game', 'all', 'were', 'win', 'against', 'two', 'one', 'there', 'her', 'last', 'she', 'up', 'would', 'new', 'its', 'if', 'before', 'also', 'you', 'back', 'time', 'my', 'team', 'play', 'can', 'which', 'second', 'three', 'players', 'test', 'just', 'off', 'only', 'match', 'set', 'year', 'more', 'final', 'cup', 'good', 'very', 'now', 'six', 'so', 'him', 'side', 'into', 'well', 'then', 'four', 'about', 'could', 'made', 'wales', 'years', 'some']


In [12]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler

# Fonction pour convertir les textes en vecteurs de mots
def document_vector(model, doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv] or [np.zeros(embedding_dim)], axis=0)


for technique, model in techniques.items():
    if technique == "Word Embeddings":
        X_train_tokens = [simple_preprocess(text) for text in X_train]
        X_test_tokens = [simple_preprocess(text) for text in X_test]

        # Entraînement du modèle Word2Vec
        embedding_dim = 100
        word2vec_model = Word2Vec(sentences=X_train_tokens, vector_size=embedding_dim, window=5, min_count=2, workers=4)

        train_x = np.array([document_vector(word2vec_model, doc) for doc in X_train_tokens])
        test_x = np.array([document_vector(word2vec_model, doc) for doc in X_test_tokens])


    elif technique == "NLP based features":
        continue

    
    elif technique == "Prédicteurs de base sélectionnés":
        technique == "Prédicteurs de base sélectionnés"
        print(50*"=")
        print(f"Technique: {technique}")

        X_base = X_rfe

        print(f"Shape:",X_base.shape)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_base)
        X = X_scaled
        y = df['category_encoded']  # Les labels encodés
        
        train_x, test_x, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
    elif model is None:
        continue

    else:
        print(50*"=")
        print(f"Technique: {technique}")
        train_x = model.fit_transform(X_train)
        test_x = model.transform(X_test)

    appliquer_model_classification(technique, train_x, test_x)
    print("\n")

    

Technique: TfidfVectorizer

Comparaison des scores F1 et Accuracy:
Logistic Regression: F1-score = 0.9924, Accuracy = 0.9925
Random Forest: F1-score = 0.9962, Accuracy = 0.9962
SVM: F1-score = 0.9887, Accuracy = 0.9887
KNN: F1-score = 0.9583, Accuracy = 0.9585


Technique: Bag of Words

Comparaison des scores F1 et Accuracy:
Logistic Regression: F1-score = 0.9962, Accuracy = 0.9962
Random Forest: F1-score = 0.9925, Accuracy = 0.9925
SVM: F1-score = 0.9450, Accuracy = 0.9434
KNN: F1-score = 0.8115, Accuracy = 0.8151



Comparaison des scores F1 et Accuracy:
Logistic Regression: F1-score = 0.7943, Accuracy = 0.7962
Random Forest: F1-score = 0.8902, Accuracy = 0.8906
SVM: F1-score = 0.6463, Accuracy = 0.6453
KNN: F1-score = 0.7740, Accuracy = 0.7736


Technique: Prédicteurs de base

Comparaison des scores F1 et Accuracy:
Logistic Regression: F1-score = 0.9924, Accuracy = 0.9925
Random Forest: F1-score = 0.9925, Accuracy = 0.9925
SVM: F1-score = 0.9339, Accuracy = 0.9321
KNN: F1-score = 0.

In [13]:
df_pivot = df_metrics.pivot_table(index='Technique', columns='Modele', values=['Score F1', 'Accuracy'])
df_pivot

Accuracy                                    \
Modele                                 KNN Logistic Regression Random Forest   
Technique                                                                      
Bag of Words                      0.815094            0.996226      0.992453   
Prédicteurs de base               0.830189            0.992453      0.992453   
Prédicteurs de base sélectionnés  0.784906            0.788679      0.890566   
TfidfVectorizer                   0.958491            0.992453      0.996226   
Word Embeddings                   0.773585            0.796226      0.890566   

                                            Score F1                      \
Modele                                 SVM       KNN Logistic Regression   
Technique                                                                  
Bag of Words                      0.943396  0.811519            0.996225   
Prédicteurs de base               0.932075  0.828524            0.992416   
Prédicteurs de base sélectionnés  0.803774  0.780295            0.787898   
TfidfVectorizer                   0.988679  0.958316            0.992419   
Word Embeddings                   0.645283  0.773970            0.794276   

                                                          
Modele                           Random Forest       SVM  
Technique                                                 
Bag of Words                          0.992451  0.945030  
Prédicteurs de base                   0.992453  0.933889  
Prédicteurs de base sélectionnés      0.890499  0.803281  
TfidfVectorizer                       0.996224  0.988740  
Word Embeddings                       0.890191  0.646263

In [14]:
end_time = time.time()
    
# Calcul du temps écoulé
execution_time = end_time - start_time
    
print(f"Temps d'exécution: {execution_time:.6f} secondes")

Temps d'exécution: 300.230490 secondes
